**GEREKLİ KÜTÜPHANELERİN YÜKLENMESİ**

In [ ]:
# pytesseract + python + collab

!pip install pytesseract
!pip install pillow
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

import pytesseract
print(pytesseract.get_tesseract_version())
print(pytesseract.get_languages())
from PIL import ImageTk, Image
from pytesseract import Output
import os
import cv2
import argparse
import csv

%cd /usr/share/tesseract-ocr/4.00/tessdata/      

from google.colab import drive
drive.mount('/content/drive')


**DATASETTE TAHMİN YAP (PYTESSERACT KULLANARAK) CSV'YE YAZDIR**

In [ ]:
directory="/content/drive/MyDrive/digital/"
row_list = []

for file in os.listdir(directory):
  
  if file.endswith(".jpg") or file.endswith(".png"): 

    filename = directory + file

    results = str(pytesseract.image_to_string(Image.open(filename), output_type=Output.STRING,lang="original", config=" --psm 8 --oem 3 -c tessedit_char_whitelist=0123456789"))
    
    # Make a list of the filename and the read value
    row = [file, results]
    
    # Append the filename and the read value of the meter as a row
    
    row_list.append(row)
    
    print(file)
    print("------ Sayac Miktari: ",  results)

# Create a new csv file and write the values to it
with open('/content/drive/MyDrive/digital/outputs.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerows(row_list)


**TEK SAYAÇ İÇİN SONUÇ TAHMİNİ (pytesseract)**

In [ ]:
results = str(pytesseract.image_to_string(Image.open("/content/21.png"), output_type=Output.STRING,lang="original", config=" --psm 13 --oem 3 -c tessedit_char_whitelist=0123456789"))
print("prediction:" , results)
# bütün fotoyu text line gibi oku -> psm 13
#result2 = str(pytesseract.image_to_data(Image.open("/content/19.jpg"), lang="original", config=" --psm 8 --oem 3 -c tessedit_char_whitelist=0123456789"))
#print("data:" , result2)

prediction: 224865



**TAHMİNLERİ OKU (MATLAB OCR) VE DF OLUŞTUR** 

In [ ]:
import pandas as pd

# results.txt -> matlab klasörü içerisinde.
# digital_actual_meter yüklenmeli.

read_value = pd.read_csv('results.txt', delimiter = ",", header=None)
read_value.columns = ["imageID", "read_value"]
read_value

,imageID,read_value
0,1.png,20208302
1,2.png,88366358
2,3.png,03000800633
3,4.png,88438393
4,5.png,004886 16
5,6.png,05056555
6,7.png,86425244
7,8.png,002144
8,9.png,0088000965
9,10.png,00008015


**GERÇEK DEĞERLERİ OKU (MATLAB OCR) VE DF OLUŞTUR** 

In [ ]:
import numpy as np

actual_value = pd.read_excel("digital_actual_meter.xlsx", dtype={'real_value':np.str})
actual_value

,imageID,real_value
0,1.png,20200302
1,2.png,00366350
2,3.png,00000002
3,4.png,00438393
4,5.png,00488516
5,6.png,03751353
6,7.png,06429244
7,8.png,002144
8,9.png,000000365
9,10.png,00000015


**MERGE**

In [ ]:
merged = pd.merge(read_value, actual_value, on="imageID") # merge dataframes
%load_ext google.colab.data_table 
merged

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,imageID,read_value,real_value
0,1.png,20208302,20200302
1,2.png,88366358,00366350
2,3.png,03000800633,00000002
3,4.png,88438393,00438393
4,5.png,004886 16,00488516
5,6.png,05056555,03751353
6,7.png,86425244,06429244
7,8.png,002144,002144
8,9.png,0088000965,000000365
9,10.png,00008015,00000015


**IFMATCH**

In [ ]:
merged['IfMatch'] = np.where(merged['read_value'] == merged['real_value'], 'True', 'False')
merged[merged['IfMatch'] == 'True']

,imageID,read_value,real_value,IfMatch
7,8.png,002144,002144,True
18,19.png,41320158,41320158,True
19,20.png,22465,22465,True


**CONVERT VALUES TO STRING && REMOVE WHITESPACES FROM PREDICTIONS**

In [ ]:
merged["read_value"] = merged["read_value"].astype(str)

merged.read_value = merged.read_value.str.replace(' ', '')

merged["real_value) "] = merged["real_value"].astype(str)

**DIGIT ACCURACY**

In [ ]:
merged["Total"] = merged['read_value'] 

actual_digit_counter = 0

for row in range(24):
  match_counter = 0

  read_digit = len(merged["read_value"][row])
  actual_digit = len(merged["real_value"][row])

  actual_digit_counter += actual_digit

  if read_digit == actual_digit:

    for basamak in range(read_digit):

      if merged['read_value'][row][basamak]== merged['real_value'][row][basamak]:
          match_counter += 1


  elif read_digit > actual_digit:
    
    for basamak in range(actual_digit):

      if merged['read_value'][row][basamak]== merged['real_value'][row][basamak]:
          match_counter += 1

  elif read_digit < actual_digit:

    for basamak in range(read_digit):

      if merged['read_value'][row][basamak]== merged['real_value'][row][basamak]:
          match_counter += 1


  merged['Total'][row] = match_counter

**ACCURACY**

In [ ]:
pd.options.mode.chained_assignment = None 

merged["Rakam Bazinda Dogru Okuma Orani"] = 0

for row in range(24):

  actual_digit = len(merged["real_value"][row])
  merged["Rakam Bazinda Dogru Okuma Orani"][row] = (merged["Total"][row]/actual_digit)*100


print("accuracy" , (sum (merged["Total"])/ (actual_digit_counter))*100)

merged

accuracy 71.80851063829788


,imageID,read_value,real_value,IfMatch,real_value),Total,Rakam Bazinda Dogru Okuma Orani
0,1.png,20208302,20200302,False,20200302,7,87
1,2.png,88366358,00366350,False,00366350,5,62
2,3.png,03000800633,00000002,False,00000002,5,62
3,4.png,88438393,00438393,False,00438393,6,75
4,5.png,00488616,00488516,False,00488516,7,87
5,6.png,05056555,03751353,False,03751353,3,37
6,7.png,86425244,06429244,False,06429244,6,75
7,8.png,002144,002144,True,002144,6,100
8,9.png,0088000965,000000365,False,000000365,4,44
9,10.png,00008015,00000015,False,00000015,7,87
